### **Integrando Word2Vec**


#### **Configuración**


Para este cuaderno, utilizarás las siguientes librerías:

*   [`torch`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) para construir modelos de redes neuronales y preparar los datos.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) para operaciones matemáticas.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) para visualizar los datos.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) para herramientas adicionales de graficación.


#### **Instalación de librerías requeridas**
<h5 style="color:red;">Después de instalar las siguientes librerías, por favor REINICIA EL KERNEL y ejecuta todas las celdas.</h5>


In [ ]:
# Todas las librerías requeridas para este cuaderno se enumeran a continuación.
#!mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Nota: Si tu entorno no soporta "!mamba install", utiliza "!pip install"

In [ ]:
#!pip install gensim #4.2.0
#!pip install portalocker>=2.0.0
#!pip install -Uq torchtext #
#!pip install -Uq torch
#!pip install -Uq torchdata


#### Importación de librerías requeridas

_Se recomienda que importes todas las librerías necesarias en un solo lugar (aquí):_


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

from IPython.core.display import display, SVG


from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset

import logging
#from gensim.models import Word2Vec
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe, vocab
from torchdata.datapipes.iter import IterableWrapper, Mapper
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm

%matplotlib inline

# También puedes usar esta sección para suprimir advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


Se define una función para graficar los embeddings de palabras en un espacio 2D.


In [ ]:
def plot_embeddings(word_embeddings, vocab=vocab):

    tsne = TSNE(n_components=2, random_state=0)
    word_embeddings_2d = tsne.fit_transform(word_embeddings)

    # Graficar los resultados con las etiquetas del vocabulario
    plt.figure(figsize=(15, 15))
    for i, word in enumerate(vocab.get_itos()):  # asumiendo que vocab.itos devuelve la lista de palabras en tu vocabulario
        plt.scatter(word_embeddings_2d[i, 0], word_embeddings_2d[i, 1])
        plt.annotate(word, (word_embeddings_2d[i, 0], word_embeddings_2d[i, 1]))

    plt.xlabel("componente t-SNE 1")
    plt.ylabel("componente t-SNE 2")
    plt.title("Embeddings de palabras visualizados con t-SNE")
    plt.show()

Se define una función que retorne palabras similares a una palabra específica calculando la distancia coseno.


In [ ]:
# Esta función retorna las palabras más similares a una palabra objetivo calculando la distancia coseno entre vectores
def find_similar_words(word, word_embeddings, top_k=5):
    if word not in word_embeddings:
        print("Palabra no encontrada en los embeddings.")
        return []

    # Obtén el embedding para la palabra dada
    target_embedding = word_embeddings[word]

    # Calcula las similitudes coseno entre la palabra objetivo y todas las demás
    similarities = {}
    for w, embedding in word_embeddings.items():
        if w != word:
            similarity = torch.dot(target_embedding, embedding) / (
                torch.norm(target_embedding) * torch.norm(embedding)
            )
            similarities[w] = similarity.item()

    # Ordena las similitudes en orden descendente
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Retorna las top k palabras similares
    most_similar_words = [w for w, _ in sorted_similarities[:top_k]]
    return most_similar_words


Se define una función que entrene el modelo word2vec con datos simples.


In [ ]:
def train_model(modelo, dataloader, criterion, optimizer, num_epochs=1000):
    """
    Entrena el modelo durante el número especificado de épocas.
    
    Args:
        modelo: El modelo de PyTorch a entrenar.
        dataloader: DataLoader que proporciona los datos para entrenamiento.
        criterion: Función de pérdida.
        optimizer: Optimizador para actualizar los pesos del modelo.
        num_epochs: Número de épocas para entrenar el modelo.

    Returns:
        modelo: El modelo entrenado.
        epoch_losses: Lista de pérdidas promedio para cada época.
    """
    
    # Lista para almacenar la pérdida en cada época
    epoch_losses = []

    for epoch in tqdm(range(num_epochs)):
        # Almacena la pérdida acumulada para la época actual
        running_loss = 0.0

        # Uso de tqdm para una barra de progreso
        for idx, samples in enumerate(dataloader):

            optimizer.zero_grad()
            
            # Comprueba si el modelo contiene una capa EmbeddingBag
            if any(isinstance(module, nn.EmbeddingBag) for _, module in modelo.named_modules()):
                target, context, offsets = samples
                predicted = modelo(context, offsets)
            
            # Comprueba si el modelo contiene una capa Embedding
            elif any(isinstance(module, nn.Embedding) for _, module in modelo.named_modules()):
                target, context = samples
                predicted = modelo(context)
                
            loss = criterion(predicted, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.1)
            optimizer.step()
            running_loss += loss.item()

        # Agrega la pérdida promedio de la época
        epoch_losses.append(running_loss / len(dataloader))
    
    return modelo, epoch_losses


#### **Word2Vec**

Word2Vec es una familia de métodos que transforma palabras en vectores numéricos, posicionando palabras similares cerca unas de otras en un espacio definido por estos números. De esta manera, puedes cuantificar y analizar las relaciones entre palabras matemáticamente. Por ejemplo, palabras como "cat" y "kitten" o "cat" y "dog" tienen vectores que están cercanos, mientras que una palabra como "book" se posiciona más lejos en este espacio vectorial.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Words.png" alt="Ejemplo de Word2Vec" class="bg-primary" width="400px">



#### **GloVe**

GloVe, por otro lado, es otro algoritmo popular para aprender embeddings de palabras.  A diferencia de word2vec, que se basa en predecir palabras de contexto/objetivo, GloVe se enfoca en capturar las estadísticas globales de co-ocurrencia de palabras en todo el corpus. Construye una matriz de co-ocurrencia que representa la frecuencia con la que las palabras aparecen juntas en el texto. La matriz luego se factoriza para obtener los embeddings de palabras. Por ejemplo, si "Man" y "King" coocurren muchas veces, sus vectores serán similares.

El modelo GloVe sigue un enfoque fundamental al construir una gran matriz de co-ocurrencia de palabra-contexto que contiene pares de `(palabra, contexto)`. Cada entrada en esta matriz representa la frecuencia con la que una palabra aparece dentro de un contexto dado, que puede ser una secuencia de palabras. El objetivo del modelo es utilizar técnicas de factoración de matrices para aproximar esta matriz de co-ocurrencia. El proceso se ilustra en el siguiente diagrama:

1. Crear una matriz de co-ocurrencia de palabra-contexto: El modelo comienza generando una matriz que captura la información de co-ocurrencia de las palabras y sus contextos circundantes. Cada elemento de la matriz representa cuán a menudo un par específico de palabra y contexto coocurre en los datos de entrenamiento.

2. Aplicar factoración de matrices: A continuación, el modelo GloVe aplica métodos de factoración de matrices para aproximar la matriz de co-ocurrencia de palabra-contexto. El objetivo es descomponer la matriz original en representaciones de dimensión inferior que capturen las relaciones semánticas entre palabras y contextos.

3. Obtener embeddings de palabra y contexto: Al factorizar la matriz de co-ocurrencia, el modelo obtiene embeddings para palabras y contextos. Estos embeddings son representaciones numéricas que codifican el significado semántico y las relaciones de las palabras y los contextos.

Para lograr esto, generalmente puedes comenzar inicializando WF (Word-Feature matrix) y FC (Feature-Context matrix) con pesos aleatorios. Luego, realizarás una operación de multiplicación entre estas matrices para obtener WC' (una aproximación de WC), y evaluar su similitud con WC. Este proceso se repite varias veces utilizando el descenso de gradiente estocástico (SGD) para minimizar el error (WC' - WC).

Una vez completado el entrenamiento, la matriz resultante WF  proporciona embeddings de palabras o representaciones vectoriales para cada palabra (el vector verde en el diagrama). La dimensionalidad de los vectores de embedding se puede determinar previamente estableciendo el valor de `F` a un número específico de dimensiones, lo que permite una representación compacta de la semántica de las palabras.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/matrix%20fact.png" alt="Matriz de co-ocurrencia" class="bg-primary" width="600px">

La principal ventaja de GloVe es que puede incorporar tanto estadísticas globales como información de contexto local. Esto resulta en embeddings de palabras que no solo capturan las relaciones semánticas entre palabras, sino que también preservan ciertas relaciones sintácticas.


#### **Crear y entrenar modelos word2vec**



In [ ]:
toy_data = """I wish I was little bit taller
I wish I was a baller
She wore a small black dress to the party
The dog chased a big red ball in the park
He had a huge smile on his face when he won the race
The tiny kitten played with a fluffy toy mouse
The team celebrated their victory with a grand parade
She bought a small, delicate necklace for her sister
The mountain peak stood majestic and tall against the clear blue sky
The toddler took small, careful steps as she learned to walk
The house had a spacious backyard with a big swimming pool
He felt a sense of accomplishment after completing the challenging puzzle
The chef prepared a delicious, flavorful dish using fresh ingredients
The children played happily in the small, cozy room
The book had an enormous impact on readers around the world
The wind blew gently, rustling the leaves of the tall trees
She painted a beautiful, intricate design on the small canvas
The concert hall was filled with thousands of excited fans
The garden was adorned with colorful flowers of all sizes
I hope to achieve great success in my chosen career path
The skyscraper towered above the city, casting a long shadow
He gazed in awe at the breathtaking view from the mountaintop
The artist created a stunning masterpiece with bold brushstrokes
The baby took her first steps, a small milestone that brought joy to her parents
The team put in a tremendous amount of effort to win the championship
The sun set behind the horizon, painting the sky in vibrant colors
The professor gave a fascinating lecture on the history of ancient civilizations
The house was filled with laughter and the sound of children playing
She received a warm, enthusiastic welcome from the audience
The marathon runner had incredible endurance and determination
The child's eyes sparkled with excitement upon opening the gift
The ship sailed across the vast ocean, guided by the stars
The company achieved remarkable growth in a short period of time
The team worked together harmoniously to complete the project
The puppy wagged its tail, expressing its happiness and affection
She wore a stunning gown that made her feel like a princess
The building had a grand entrance with towering columns
The concert was a roaring success, with the crowd cheering and clapping
The baby took a tiny bite of the sweet, juicy fruit
The athlete broke a new record, achieving a significant milestone in her career
The sculpture was a masterpiece of intricate details and craftsmanship
The forest was filled with towering trees, creating a sense of serenity
The children built a small sandcastle on the beach, their imaginations running wild
The mountain range stretched as far as the eye could see, majestic and awe-inspiring
The artist's brush glided smoothly across the canvas, creating a beautiful painting
She received a small token of appreciation for her hard work and dedication
The orchestra played a magnificent symphony that moved the audience to tears
The flower bloomed in vibrant colors, attracting butterflies and bees
The team celebrated their victory with a big, extravagant party
The child's laughter echoed through the small room, filling it with joy
The sunflower stood tall, reaching for the sky with its bright yellow petals
The city skyline was dominated by tall buildings and skyscrapers
The cake was adorned with a beautiful, elaborate design for the special occasion
The storm brought heavy rain and strong winds, causing widespread damage
The small boat sailed peacefully on the calm, glassy lake
The artist used bold strokes of color to create a striking and vivid painting
The couple shared a passionate kiss under the starry night sky
The mountain climber reached the summit after a long and arduous journey
The child's eyes widened in amazement as the magician performed his tricks
The garden was filled with the sweet fragrance of blooming flowers
The basketball player made a big jump and scored a spectacular slam dunk
The cat pounced on a small mouse, displaying its hunting instincts
The mansion had a grand entrance with a sweeping staircase and chandeliers
The raindrops fell gently, creating a rhythmic patter on the roof
The baby took a big step forward, encouraged by her parents' applause
The actor delivered a powerful and emotional performance on stage
The butterfly fluttered its delicate wings, mesmerizing those who watched
The company launched a small-scale advertising campaign to test the market
The building was constructed with strong, sturdy materials to withstand earthquakes
The singer's voice was powerful and resonated throughout the concert hall
The child built a massive sandcastle with towers, moats, and bridges
The garden was teeming with a variety of small insects and buzzing bees
The athlete's muscles were well-developed and strong from years of training
The sun cast long shadows as it set behind the mountains
The couple exchanged heartfelt vows in a beautiful, intimate ceremony
The dog wagged its tail vigorously, a sign of excitement and happiness
The baby let out a tiny giggle, bringing joy to everyone around"""


A continuación, se prepara los datos a tokenizar y se crea un vocabulario a partir de ellos.


In [ ]:
# Paso 1: Obtener tokenizador
tokenizer = get_tokenizer('basic_english')  # Esto usa el tokenizador básico en inglés. Puedes elegir otro.

# Paso 2: Tokenizar oraciones
def tokenize_data(sentences):
    for sentence in sentences:
        yield tokenizer(sentence)

tokenized_toy_data = tokenizer(toy_data)


vocab = build_vocab_from_iterator(tokenize_data(tokenized_toy_data), specials=['<unk>'])
vocab.set_default_index(vocab["<unk>"])


Veamos cómo se ve una oración después de la tokenización y de la numeración:


In [ ]:
# Prueba
sample_sentence = "I wish I was  a baller"
tokenized_sample = tokenizer(sample_sentence)
encoded_sample = [vocab[token] for token in tokenized_sample]
print("Muestra codificada:", encoded_sample)

Se escribe una función para aplicar la numeración a todos los tokens:


In [ ]:
text_pipeline = lambda tokens: [vocab[token] for token in tokens]

#### **Bolsa continua de palabras (CBOW)**

Para el modelo CBOW, se utiliza un "contexto" para predecir una palabra objetivo. El "contexto" es típicamente un conjunto de palabras circundantes. Por ejemplo, si una ventana de contexto es de tamaño 2, entonces se toma dos palabras antes y dos palabras después de la palabra objetivo como contexto. La palabra objetivo se muestra en rojo y el contexto en azul:


<table border="1">
    <tr>
        <th>Paso de tiempo</th>
        <th>Frase</th>
    </tr>
    <tr>
        <td>1</td>
        <td><span style="color:blue;">I wish</span> <span style="color:red;">I</span> <span style="color:blue;">was  little </span></td>
    </tr>
    <tr>
        <td>2</td>
        <td><span style="color:blue;">wish I</span> <span style="color:red;">was</span> <span style="color:blue;">little bit </span></td>
    </tr>
    <tr>
        <td>3</td>
        <td><span style="color:blue;">I was</span> <span style="color:red;">little</span> <span style="color:blue;">  bit taller</span></td>
    </tr>
    <tr>
        <td>4</td>
        <td><span style="color:blue;">was little</span> <span style="color:red;">bit</span> <span style="color:blue;"> taller I</span></td>
    </tr>
    <tr>
        <td>5</td>
        <td><span style="color:blue;">little bit</span> <span style="color:red;">taller</span> <span style="color:blue;"> I wish</span></td>
    </tr>
    <tr>
        <td>6</td>
        <td><span style="color:blue;">bit taller</span> <span style="color:red;">I</span> <span style="color:blue;">wish I</span></td>
    </tr>
    <tr>
        <td>7</td>
        <td><span style="color:blue;">taller I</span> <span style="color:red;">wish</span> <span style="color:blue;">I was</span></td>
    </tr>
    <tr>
        <td>8</td>
        <td><span style="color:blue;">I wish</span> <span style="color:red;">I</span> <span style="color:blue;">was a</span></td>
    </tr>
    <tr>
        <td>9</td>
        <td><span style="color:blue;">wish I</span> <span style="color:red;">was</span> <span style="color:blue;">a baller</span></td>
    </tr>
</table>



Se puede desplazar la secuencia y crear los datos de entrenamiento:


In [ ]:
CONTEXT_SIZE = 2

cobow_data = []
for i in range(1, len(tokenized_toy_data) - CONTEXT_SIZE):
    context = (
        [tokenized_toy_data[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [tokenized_toy_data[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = tokenized_toy_data[i]
    cobow_data.append((context, target))


Imprimimos un ejemplo, mostrando tanto las palabras de contexto (por ejemplo, `['wish', 'i', 'was', 'little']`) como la palabra objetivo (por ejemplo, `'i'`):

<table border="1">
<tr>
        <th>Paso de tiempo</th>
        <th>Frase</th>
    </tr>
    <tr>
        <td>1</td>
        <td><span style="color:blue;">I wish</span> <span style="color:red;">I</span> <span style="color:blue;">was  little </span></td>
    </tr>
</table>


In [ ]:
print(cobow_data[1])

Se imprime la siguiente muestra, mostrando tanto el contexto (por ejemplo, `['i', 'wish', 'little', 'bit']`) como la palabra objetivo: `'was'`

<table border="1">
    <tr>
        <th>Paso de tiempo</th>
        <th>Frase</th>
    </tr>
    <tr>
        <td>2</td>
        <td><span style="color:blue;">wish I</span> <span style="color:red;">was</span> <span style="color:blue;"> little bit</span></td>
    </tr>
</table>


In [ ]:
print(cobow_data[2])

Se encuentra los embeddings que guíen al modelo para predecir las siguientes probabilidades. $P(w_t| w_{t-2},w_{t-1},w_{t+1},w_{t+2})$ es la probabilidad de $w_t$, la palabra objetivo, condicionada a la ocurrencia de las palabras de contexto $w_{t-2},w_{t-1},w_{t+1},w_{t+2}$.


<table border="1">
    <thead>
        <tr>
            <th>\( P(w_t| w_{t-2},w_{t-1},w_{t+1},w_{t+2}) \)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>\( P(w_1 | w_{-1},w_0,w_2,w_3) = P(I | \text{I wish, was little}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_2 | w_0,w_1,w_3,w_4) = P(was | \text{wish I, little bit}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_3 | w_1,w_2,w_4,w_5) = P(little | \text{I was, bit taller}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_4 | w_2,w_3,w_5,w_6) = P(bit | \text{was little, taller I}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_5 | w_3,w_4,w_6,w_7) = P(taller | \text{little bit, I wish}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_6 | w_4,w_5,w_7,w_8) = P(I | \text{bit taller, wish I}) \)</td>
        </tr>
        <tr>
            <td>\( P(w_7 | w_5,w_6,w_8,w_9) = P(wish | \text{taller I, I was}) \)</td>
        </tr>
    </tbody>
</table>


La función `collate_batch` procesa lotes de datos, convirtiendo el texto (contexto y palabra objetivo) a formato numérico usando el vocabulario y organizándolos para el entrenamiento del modelo.


In [ ]:
def collate_batch(batch):
    target_list, context_list, offsets = [], [], [0]
    for _context, _target in batch:
        
        target_list.append(vocab[_target])  
        processed_context = torch.tensor(text_pipeline(_context), dtype=torch.int64)
        context_list.append(processed_context)
        offsets.append(processed_context.size(0))
    target_list = torch.tensor(target_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    context_list = torch.cat(context_list)
    return target_list.to(device), context_list.to(device), offsets.to(device)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


Se selecciona las primeras 10 muestras de la lista `cobow_data` y se procesa usando la función `collate_batch`. Los resultados son las palabras objetivo tokenizadas (`target_list`), las palabras de contexto (`context_list`) y los índices iniciales de contexto para cada muestra (`offsets`).


In [ ]:
target_list, context_list, offsets = collate_batch(cobow_data[0:10])
print(f"target_list (palabras objetivo tokenizadas): {target_list} , context_list (palabras de contexto): {context_list} , offsets (índices de inicio para cada contexto): {offsets}")


Se crea un objeto ```DataLoader```:


In [ ]:
BATCH_SIZE = 64  # tamaño del lote para entrenamiento

dataloader_cbow = DataLoader(
    cobow_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
print(dataloader_cbow) 

El modelo CBOW que se muestra a continuación comienza con una capa `EmbeddingBag`, que toma una lista de índices de palabras de contexto (de longitud variable) y produce el promedio de los embeddings de dichas palabras. Este embedding se pasa luego a través de una capa lineal que reduce su dimensión a `embed_dim/2`. 

Tras aplicar la activación ReLU, la salida se procesa con otra capa lineal, transformándola para que coincida con el tamaño del vocabulario, permitiendo así que el modelo prediga la probabilidad de cualquier palabra del vocabulario como palabra objetivo. 
El flujo general va desde los índices de las palabras de contexto hasta predecir la palabra central en el enfoque de CBOW.


In [ ]:
class CBOW(nn.Module):
    # Inicializar el modelo CBOW
    def __init__(self, vocab_size, embed_dim, num_class):
        
        super(CBOW, self).__init__()
         # Definir la capa de embedding usando nn.EmbeddingBag
        # Esta capa produce el promedio de los embeddings de las palabras de contexto
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        # Definir la primera capa lineal con tamaño de entrada embed_dim y tamaño de salida embed_dim//2
        self.linear1 = nn.Linear(embed_dim, embed_dim//2)
        # Definir la capa completamente conectada con tamaño de entrada embed_dim//2 y tamaño de salida vocab_size
        self.fc = nn.Linear(embed_dim//2, vocab_size)
        

        self.init_weights()
    # Inicializa los pesos de los parámetros del modelo
    def init_weights(self):
        # Inicializa los pesos de la capa de embedding
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        # Inicializa los pesos de la capa completamente conectada
        self.fc.weight.data.uniform_(-initrange, initrange)
        # Inicializa los sesgos de la capa completamente conectada a ceros
        self.fc.bias.data.zero_()
        

    def forward(self, texto, offsets):
        # Pasa el texto de entrada y los offsets a través de la capa de embedding
        out = self.embedding(texto, offsets)
        # Aplica la función de activación ReLU a la salida de la primera capa lineal
        out = torch.relu(self.linear1(out))
        # Pasa la salida de la activación ReLU a través de la capa completamente conectada
        return self.fc(out)


Se crea una instancia del modelo CBOW:


In [ ]:
vocab_size = len(vocab)
emsize = 24
model_cbow = CBOW(vocab_size, emsize, vocab_size).to(device)


Se define la función de pérdida, el optimizador y el scheduler para el entrenamiento:


In [ ]:
LR = 5  # tasa de aprendizaje

# Define el criterio CrossEntropyLoss. Es comúnmente usado para tareas de clasificación multiclase.
# Este criterio combina la función softmax con la pérdida de log-verosimilitud negativa.
criterion = torch.nn.CrossEntropyLoss()

# Define el optimizador utilizando descenso de gradiente estocástico (SGD).
# Optimiza los parámetros del modelo model_cbow, obtenidos con model_cbow.parameters().
# La tasa de aprendizaje (lr) determina el tamaño del paso en cada actualización de los parámetros.
optimizer = torch.optim.SGD(model_cbow.parameters(), lr=LR)

# Define un scheduler para la tasa de aprendizaje.
# El scheduler StepLR ajusta la tasa de aprendizaje durante el entrenamiento.
# Multiplica la tasa de aprendizaje por gamma cada 1 época (aquí, 1.0).
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)


Se entrena el modelo:


In [ ]:
model_cbow, epoch_losses = train_model(model_cbow, dataloader_cbow, criterion, optimizer, num_epochs=400)

Graficamos los valores de pérdida a lo largo del entrenamiento:


In [ ]:
plt.plot(epoch_losses)
plt.xlabel("épocas")


Los pesos del modelo son los embeddings de palabras reales. Se pueden cargar en un arreglo de numpy:


In [ ]:
word_embeddings = model_cbow.embedding.weight.detach().cpu().numpy()


Revisamos el vector embebido para una palabra de ejemplo. Observamos la dimensión de este vector, que es igual a `emsize = 24` que se definió anteriormente.


In [ ]:
word = 'baller'
word_index = vocab.get_stoi()[word]  # obtener el índice de la palabra en el vocabulario
print(word_embeddings[word_index])


Podemos comprobar si los embeddings representan las similitudes entre las palabras. Para ello, a efectos de visualización, se necesita reducir la dimensión de los embeddings para mapear el espacio de embeddings a un espacio 2D. 



In [ ]:
plot_embeddings(word_embeddings, vocab=vocab)

Al examinar las proyecciones de t-SNE, se evidencia que, pese a la inevitable pérdida de información por la reducción de dimensiones y las limitaciones de un conjunto de datos pequeño, las palabras con significados similares se agrupan. Por ejemplo, palabras como 'bright' y 'shadow' se encuentran próximas cerca del punto (-15, -15) en los componentes 1 y 2. De igual forma, 'cat', 'dog' y 'mouse' se agrupan alrededor del punto (5, 5), y 'sailed' junto con 'wind' pueden encontrarse próximos al punto (5, -8).


#### **Modelo skip-gram**

El modelo skip-gram es una de las dos arquitecturas principales utilizadas en word2vec, una técnica popular para aprender embeddings de palabras. En el modelo Skip-gram, el objetivo es predecir las palabras circundantes (contexto) a partir de una palabra central (objetivo). La idea principal es que las palabras que aparecen en contextos similares tienden a tener significados similares. Considera este ejemplo:

**I was little bit taller,**

Suponiendo una ventana de contexto de tamaño 2, las palabras en rojo representan el contexto, mientras que la palabra resaltada en azul indica la palabra objetivo:

**<span style="color:red;"> I was</span> <span style="color:blue;">little</span> <span style="color:red;">bit taller, </span>**

##### **Ventanas en el modelo skip-gram:**

Entrenar el modelo skip-gram usando el contexto real puede ser computacionalmente costoso. Esto se debe a que implica predecir probabilidades para cada palabra del vocabulario en cada posición del contexto, a diferencia del CBOW, que solo predice las probabilidades para la palabra objetivo. Para mitigar esto, se emplean varias técnicas de aproximación.

Una técnica común es dividir el contexto completo en partes más pequeñas y predecirlas una a una. Esto no solo simplifica la tarea de predicción, sino que también ayuda a un mejor entrenamiento, ya que proporciona múltiples ejemplos de entrenamiento a partir de un único par `(contexto, objetivo)`.

##### **Usando la primera fila de la tabla como ejemplo:**

Para el ejemplo anterior, la palabra objetivo es **"little"**. El contexto completo para esta palabra objetivo es:

**I was bit taller**

En la aproximación, en lugar de usar el contexto completo para predecir la palabra objetivo, se debe dividir. En este ejemplo hay cuatro aproximaciones:

1. Aproximación 1: **I**
2. Aproximación 2: **was**
3. Aproximación 3: **bit**
4. Aproximación 4: **taller**

Para cada aproximación, el modelo skip-gram tratará de predecir la palabra objetivo "little" usando solo esa parte del contexto. Esto significa que, para la primera aproximación, el modelo intentará predecir "little" usando únicamente la palabra "I". Para la segunda, usará únicamente "was", y así sucesivamente.

En conclusión, el modelo skip-gram busca comprender las relaciones entre las palabras prediciendo el contexto a partir de una palabra dada. Técnicas de aproximación, como la ilustrada, ayudan a simplificar el proceso de entrenamiento y hacerlo más eficiente.


<table border="1">
    <tr>
        <th>Contexto completo con objetivo</th>
        <th>Palabra objetivo</th>
        <th>Contexto original del objetivo</th>
        <th>Approximacion 1</th>
        <th>Approximacion 2</th>
        <th>Approximacion 3</th>
        <th>Approximacion 4</th>
    </tr>
    <tr>
        <td><span style="color:red;"> I was</span> <span style="color:blue;">little</span> <span style="color:red;">bit taller, </span></td>
        <td>little</td>
        <td> I was bit taller,</td>
        <td>I</td>
        <td>was</td>
        <td>bit</td>
        <td>taller,</td>
    </tr>
    <tr>
        <td><span style="color:red;"> was little</span> <span style="color:blue;">bit</span> <span style="color:red;">taller, I </span></td>
        <td>bit</td>
        <td>was little taller, I</td>
        <td>was</td>
        <td>little</td>
        <td>taller,</td>
        <td>I</td>
    </tr>
    <tr>
        <td><span style="color:red;">little bit</span> <span style="color:blue;">taller,</span> <span style="color:red;">I wish </span></td>
        <td>taller,</td>
        <td>little bit I wish</td>
        <td>little</td>
        <td>bit</td>
        <td>I</td>
        <td>wish</td>
    </tr>
    <tr>
        <td><span style="color:red;"> bit taller,</span> <span style="color:blue;">I</span> <span style="color:red;">wish I </span></td>
        <td>I</td>
        <td>bit taller, wish I</td>
        <td>bit</td>
        <td>taller,</td>
        <td>wish</td>
        <td>I</td>
    </tr>
    <tr>
        <td><span style="color:red;"> taller, I</span> <span style="color:blue;">wish</span> <span style="color:red;">I was </span></td>
        <td>wish</td>
        <td>taller, I I was</td>
        <td>taller,</td>
        <td>I</td>
        <td>I</td>
        <td>was</td>
    </tr>
    <!-- More rows can be added in a similar pattern for other words in the phrase. -->
</table>


El objetivo es optimizar las probabilidades condicionales para obtener embeddings, optimizar las probabilidades condicionales para obtener embeddings de alta calidad. 

> La única diferencia con respecto al CBOW estándar es la estructura de las probabilidades condicionales $P(w_{t+j}| w_{t})$ para una ventana de $j=-2,-1,..,1,2$.


<table border="1">
    <tr>
        <th>j</th>
        <th>Palabra objetivo t=3 </th>
        <th>Palabra de contexto</th>
        <th>Probabilidad</th>
    </tr>
    <tr>
         <th>-1</th>
        <td>little</td>
        <td>was</td>
        <td> P(was | little)</td>
    </tr>
    <tr>
         <th>1</th>
        <td>little</td>
        <td>bit</td>
        <td>P(bit | little)</td>
    </tr>
    <tr>
         <th>2</th>
        <td>little</td>
        <td>taller</td>
        <td>P(taller | little) </td>
    </tr>
    <!-- Repeat rows for each context word for each target word -->
</table>


En contraste con la notación estándar en probabilidad condicional, donde la variable dependiente se representa generalmente como la objetivo, la terminología actual invierte esta convención.


Este código construye un conjunto de datos para el modelo skip-gram a partir de datos tokenizados, en el que para cada palabra (objetivo) se recogen las palabras circundantes dentro de una ventana especificada (contexto) definida por `CONTEXT_SIZE`.


In [ ]:
# Define el tamaño de la ventana para el contexto alrededor de la palabra objetivo.
CONTEXT_SIZE = 2

# Inicializa una lista vacía para almacenar los pares (objetivo, contexto).
skip_data = []

# Itera sobre cada palabra en tokenized_toy_data, excluyendo las primeras y últimas palabras según CONTEXT_SIZE.
for i in range(CONTEXT_SIZE, len(tokenized_toy_data) - CONTEXT_SIZE):

    # Para la palabra en la posición i, el contexto comprende las palabras anteriores (de acuerdo al CONTEXT_SIZE)
    # y las palabras siguientes. Se recoge el contexto en una lista.
    context = (
        [tokenized_toy_data[i - j - 1] for j in range(CONTEXT_SIZE)]  # Palabras precedentes
        + [tokenized_toy_data[i + j + 1] for j in range(CONTEXT_SIZE)]  # Palabras siguientes
    )

    # La palabra en la posición i se toma como objetivo.
    target = tokenized_toy_data[i]

    # Agrega el par (objetivo, contexto) a la lista skip_data.
    skip_data.append((target, context))


Se puede aplicar una ventana al modelo skip-gram(windowing):


In [ ]:
skip_data_ = [[(sample[0], word) for word in sample[1]] for sample in skip_data]

Y obtener pares `(objetivo, contexto)`:


In [ ]:
skip_data_flat = [item for items in skip_data_ for item in items]
skip_data_flat[8:28]


Se crea una función `collate` para numerar los pares `(objetivo, contexto)`:


In [ ]:
def collate_fn(batch):
    target_list, context_list = [], []
    for _context, _target in batch:
        
        target_list.append(vocab[_target]) 
        context_list.append(vocab[_context])
        
    target_list = torch.tensor(target_list, dtype=torch.int64)
    context_list = torch.tensor(context_list, dtype=torch.int64)
    return target_list.to(device), context_list.to(device)


In [ ]:
dataloader = DataLoader(skip_data_flat, batch_size=BATCH_SIZE, collate_fn=collate_fn)

Se revisa una muestra de lote (batch) del par `(objetivo, contexto)` después de la colación:


In [ ]:
next(iter(dataloader))


Aquí, se define la red skip-gram.

La capa de embeddings se define utilizando `nn.Embedding`, que crea embeddings para las palabras del vocabulario según el tamaño y la dimensión especificados.
La capa `fc` es una capa completamente conectada con dimensión de entrada `embed_dim` y dimensión de salida `vocab_size`.

En el método forward, el texto de entrada se pasa por la capa de embeddings para obtener los vectores de las palabras. La salida de la capa de embeddings se pasa luego por la capa fc.
Se aplica la activación ReLU a la salida de la capa fc. Finalmente, se retorna la salida.


In [ ]:
class SkipGram_Model(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(SkipGram_Model, self).__init__()
        # Definir la capa de embeddings
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim
        )
        
        # Definir la capa completamente conectada
        self.fc = nn.Linear(in_features=embed_dim, out_features=vocab_size)

    def forward(self, texto):
        # Realiza el pase forward
        # Pasa el texto de entrada por la capa de embeddings
        out = self.embeddings(texto)
        
        # Pasa la salida de la capa de embeddings por la capa fc
        # Aplica la función de activación ReLU
        out = torch.relu(out)
        out = self.fc(out)
        
        return out


Se crea una instancia del modelo:


In [ ]:
emsize = 24
model_sg = SkipGram_Model(vocab_size, emsize).to(device)


Se entrenar el modelo con datos simples:


In [ ]:
LR = 5  # tasa de aprendizaje
#BATCH_SIZE = 64  # tamaño del lote para el entrenamiento

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_sg.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)


In [ ]:
model_sg, epoch_losses = train_model(model_sg, dataloader, criterion, optimizer, num_epochs=400)

In [ ]:
plt.plot(epoch_losses)


Se puede graficar los embeddings de palabras reduciendo sus dimensiones con t-SNE:


In [ ]:
word_embeddings = model_sg.embeddings.weight.detach().cpu().numpy()
plot_embeddings(word_embeddings, vocab=vocab)

Cuando se elige entre CBOW o Skip-gram, la mejor opción suele depender de las especificidades de la tarea y de los datos. Si el conjunto de datos es pequeño pero se necesita una buena representación de palabras poco frecuentes, skip-gram podría ser la mejor opción. Si lo que más importa es la eficiencia computacional y las palabras poco frecuentes no son tan importantes, CBOW podría ser suficiente. Cabe mencionar que, para conjuntos de datos muy pequeños, las ventajas de los embeddings neurales pueden ser limitadas y métodos más simples o el uso de embeddings preentrenados podrían ser más efectivos.


#### **Aplicando embeddings de palabras preentrenados**

#### Cargar el modelo Stanford GloVe

El **transfer learning**, en particular mediante el uso de embeddings preentrenados, es una piedra angular en el NLP moderno. Este enfoque aprovecha el conocimiento adquirido en una tarea, generalmente aprendido con conjuntos de datos masivos, y lo aplica a otra tarea, a menudo más especializada. La principal ventaja es doble: se evita la necesidad de enormes recursos computacionales para aprender desde cero y se inyecta una base de entendimiento lingüístico en el modelo. Al usar embeddings que ya han capturado patrones y asociaciones complejas del lenguaje, incluso modelos con poca exposición a datos específicos pueden comportarse de manera sorprendentemente sofisticada, haciendo del transfer learning un atajo estratégico para mejorar el rendimiento en el NLP.


Veamos el modelo GloVe preentrenado de Stanford:


Se puede especificar el nombre del modelo y la dimensión del embedding: `GloVe(name='GloVe_model_name', dim=300)`


In [ ]:
# creando una instancia de la versión 6B del modelo Glove()
glove_vectors_6B = GloVe(name ='6B')  # puedes especificar el modelo con el siguiente formato: GloVe(name='840B', dim=300)


In [ ]:
# creando otra instancia de un modelo Glove() más grande
#glove_vectors_840B = GloVe()


Se debe continuar con el modelo 6B ya que es más liviano. Podemos cargar distintos modelos GloVe preentrenados desde torch utilizando ```torch.nn.Embedding.from_pretrained```. 


In [ ]:
# Carga los pesos preentrenados del modelo GloVe en una capa de embedding de PyTorch
embeddings_Glove6B = torch.nn.Embedding.from_pretrained(glove_vectors_6B.vectors, freeze=True)


Observa los vectores de embedding de este gran modelo preentrenado para las palabras de tu corpus:


Creamos un arreglo que retorne el índice de cada palabra en el vocabulario del modelo GloVe:


In [ ]:
word_to_index = glove_vectors_6B.stoi  # Mapeo del vocabulario a índices
word_to_index['team']


Obtenemos el vector de embedding para una palabra:


In [ ]:
embeddings_Glove6B.weight[word_to_index['team']]


Revisamos qué tan bien el modelo GloVe captura la similitud entre palabras:


In [ ]:
# Un arreglo de palabras de ejemplo
words = [
    "taller",
    "short",
    "black",
    "white",
    "dress",
    "pants",
    "big",
    "small",
    "red",
    "blue",
    "smile",
    "frown",
    "race",
    "stroll",
    "tiny",
    "huge",
    "soft",
    "rough",
    "team",
    "individual"
]


Se crea un diccionario de palabras y sus embeddings:


In [ ]:
embedding_dict_Glove6B = {}
for word in words:
    # Obtén el índice de la palabra para acceder a su embedding
    embedding_vector = embeddings_Glove6B.weight[word_to_index[word]]
    if embedding_vector is not None:
        # Se omitirán aquellas palabras que no se encuentren en el índice de embeddings
        embedding_dict_Glove6B[word] = embedding_vector


Con los embeddings preentrenados para las palabras de ejemplo, veamos si el modelo es capaz de capturar la similitud entre ellas calculando la distancia entre palabras:


In [ ]:
# Llama a la función para encontrar palabras similares
target_word = "small"
top_k = 2
similar_words = find_similar_words(target_word, embedding_dict_Glove6B, top_k)

# Imprime las palabras similares
print("{} palabras más similares a {}:".format(top_k, target_word), similar_words)


> Verifica que el modelo GloVe preentrenado realiza un buen trabajo capturando la similitud entre palabras.


#### **Clasificación de textos utilizando embeddings de palabras preentrenados**

Usemos los embeddings preentrenados para clasificar datos textuales en distintos temas:


Primero, debemos construir un vocabulario a partir del modelo GloVe preentrenado:


In [ ]:
from torchtext.vocab import GloVe, vocab
# Construir el vocabulario a partir de glove_vectors
# vocab(ordered_dict: Dict, min_freq: int = 1, specials: Optional[List[str]] = None)
vocab = vocab(glove_vectors_6B.stoi, 0, specials=('<unk>', '<pad>'))
vocab.set_default_index(vocab["<unk>"])


In [ ]:
vocab(["<unk>", "Hello", "hello"])


A continuación, debemos tokenizar el texto. Para ello podemos usar tokenizadores preentrenados de torch:


In [ ]:
# Definir tokenizador

tokenizer = get_tokenizer("basic_english")
# Definir funciones para procesar el texto y las etiquetas


Se crean particiones (splits) del conjunto de datos `AG_NEWS()` para entrenamiento, validación y prueba:


In [ ]:
# Divide el conjunto de datos en iteradores de entrenamiento y prueba.
train_iter, test_iter = AG_NEWS()

# Convierte los iteradores de entrenamiento y prueba a datasets de estilo mapa.
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Determina el número de muestras a usar para entrenamiento y validación (5% para validación).
num_train = int(len(train_dataset) * 0.85)

# Separa aleatoriamente el dataset de entrenamiento en conjuntos de entrenamiento y validación usando `random_split`.
# El conjunto de entrenamiento contendrá el 85% de las muestras y el de validación el 15% restante.
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])


Se definen las etiquetas de clase:


In [ ]:
# Definir etiquetas de clase
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
'''ag_news_label[y]'''
num_class = len(set([label for (label, text) in train_iter]))


Agrupamos los datos en lotes (`collate data`):


In [ ]:
def text_pipeline(x):
    x = x.lower()  # se necesita porque el vocabulario está en minúsculas
    return vocab(tokenizer(x))

def label_pipeline(x):
    return int(x) - 1

# Crea la etiqueta, el texto y los offsets para cada lote de datos
# 'text' es el texto concatenado de todos los datos del lote
# necesitas los offsets (índices finales) para separar los textos y predecir sus etiquetas
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


Se crea los dataLoaders para las particiones de entrenamiento, validación y prueba:


In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


In [ ]:
label, text, offsets = next(iter(train_dataloader))
print(label, text, offsets)
label.shape, text.shape, offsets.shape


Se crea el modelo clasificador:


In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(glove_vectors_6B.vectors, freeze=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text)
        # Obtienes el promedio de los embeddings de las palabras en el texto
        means = []
        for i in range(1, len(offsets)):
            text_tmp = embedded[offsets[i-1]:offsets[i]]
            means.append(text_tmp.mean(0))

        return self.fc(torch.stack(means))


Se define una función de evaluación para calcular la precisión del modelo:


In [ ]:
def evaluate(dataloader):
    modelo.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = modelo(text, offsets)

            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count


Se crea una instancia del modelo y se comprueba su capacidad de predicción antes de entrenar:


In [ ]:
# Definir hiperparámetros
vocab_size = len(vocab)
embedding_dim = 300
# Inicializar el modelo
modelo = TextClassificationModel(vocab_size, embedding_dim, num_class).to(device)


In [ ]:
evaluate(test_dataloader)


Ahora se entrena el modelo:


In [ ]:
def train_TextClassification(modelo, dataloader, criterion, optimizer, epochs=10):
    
    cum_loss_list = []
    acc_epoch = []
    acc_old = 0

    for epoch in tqdm(range(1, EPOCHS + 1)):
        modelo.train()
        cum_loss = 0
        for idx, (label, text, offsets) in enumerate(train_dataloader):
            optimizer.zero_grad()
            
            predicted_label = modelo(text, offsets)
            
            loss = criterion(predicted_label, label)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.1)
            optimizer.step()
            cum_loss += loss.item()

        cum_loss_list.append(cum_loss / len(train_dataloader))
        accu_val = evaluate(valid_dataloader)
        acc_epoch.append(accu_val)

        if accu_val > acc_old:
            acc_old = accu_val
            torch.save(modelo.state_dict(), 'model.pth')
            
    return modelo, cum_loss_list, acc_epoch


In [ ]:
# Define hiperparámetros
LR = 0.1
EPOCHS = 10

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelo.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

modelo, cum_loss_list, acc_epoch = train_TextClassification(modelo, train_dataloader, criterion, optimizer, EPOCHS)


Se grafica la pérdida y la precisión del modelo entrenado:


In [ ]:
import matplotlib.pyplot as plt
def plot(COST, ACC):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('época', color=color)
    ax1.set_ylabel('pérdida total', color=color)
    ax1.tick_params(axis='y', color=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('precisión', color=color)  # ya se ha definido la etiqueta del eje x con ax1
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()  # para que la etiqueta del eje y derecho no se corte

    plt.show()


In [ ]:
plot(cum_loss_list, acc_epoch)


Finalmente, se evalúa el modelo en el conjunto de datos de prueba:


In [ ]:
evaluate(test_dataloader)
